## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

#### Now you are ready to start!

In [1]:
###Install the correct libraries via pip or conda then run this part of the code

In [3]:
#Name: PRISMA2GeoTIFF.py
#Description: reads he5 PRISMA files content and converts it to a GeoTIFF.
#All 66 VNIR bands and 173 SWIR bands are converted in one single GeoTIFF file.
#input is a PRISMA he5 file and output is a GeoTIFF with the same name in the same path
#Author: martin rapilly, mrapilly60@uasd.edu.do/martin.rapilly@get.omp.eu

#import libraries
import h5py
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from itertools import chain
from osgeo import gdal, osr
import os

In [3]:
###Run this code to initiate the function PRISMA2GeoTIFF

In [4]:

def PRISMA2GeoTIFF(filename):

    def first_existing(h5f, paths):
        for p in paths:
            if p in h5f:
                return p
        return None

    with h5py.File(filename, "r") as h5f:

        # --- detect which PRISMA swath exists (prefer L2D if present) ---
        swath = first_existing(h5f, [
            "HDFEOS/SWATHS/PRS_L2D_HCO",
            "HDFEOS/SWATHS/PRS_L2D_HRC",
            "HDFEOS/SWATHS/PRS_L1_HCO",
            "HDFEOS/SWATHS/PRS_L1_HRC",
        ])
        if swath is None:
            raise KeyError("No PRISMA swath found under HDFEOS/SWATHS (expected PRS_L1_* or PRS_L2D_*).")

        # groups
        data_fields = first_existing(h5f, [f"{swath}/Data Fields", f"{swath}/Data_Fields"])
        geo_fields  = first_existing(h5f, [f"{swath}/Geolocation Fields", f"{swath}/Geolocation_Fields"])
        if data_fields is None:
            raise KeyError(f"Found swath {swath} but no Data Fields group.")
        if geo_fields is None:
            raise KeyError(f"Found swath {swath} but no Geolocation Fields group.")

        # cubes
        swir_path = first_existing(h5f, [f"{data_fields}/SWIR_Cube"])
        vnir_path = first_existing(h5f, [f"{data_fields}/VNIR_Cube"])
        if swir_path is None or vnir_path is None:
            raise KeyError(f"Missing VNIR/SWIR cubes. Available: {list(h5f[data_fields].keys())}")

        # --- geolocation: L2D uses Latitude/Longitude; L1 may use *_VNIR / *_SWIR ---
        lat_path = first_existing(h5f, [
            f"{geo_fields}/Latitude",
            f"{geo_fields}/Latitude_VNIR",
            f"{geo_fields}/Latitude_SWIR",
        ])
        lon_path = first_existing(h5f, [
            f"{geo_fields}/Longitude",
            f"{geo_fields}/Longitude_VNIR",
            f"{geo_fields}/Longitude_SWIR",
        ])
        if lat_path is None or lon_path is None:
            raise KeyError(
                f"No usable Latitude/Longitude found. Available: {list(h5f[geo_fields].keys())}"
            )

        # read arrays
        SWIRcube = h5f[swir_path][()]   # expected shape often (rows, bands, cols) but may vary
        VNIRcube = h5f[vnir_path][()]
        lat = h5f[lat_path][()]
        lon = h5f[lon_path][()]

        print("Using swath:", swath)
        print("SWIR path:", swir_path, "shape:", SWIRcube.shape)
        print("VNIR path:", vnir_path, "shape:", VNIRcube.shape)
        print("LAT/LON paths:", lat_path, lon_path, "shapes:", lat.shape, lon.shape)

        # --- normalize cube layout to (bands, rows, cols) ---
        # Your original code assumes cube indexed like VNIRcube[row][band] returning a row vector (cols).
        # That corresponds to VNIRcube shape (rows, bands, cols).
        # We'll handle both common cases: (rows, bands, cols) or (bands, rows, cols)
        def to_brc(cube):
            if cube.ndim != 3:
                raise ValueError(f"Expected 3D cube, got shape {cube.shape}")
            # if first dim equals lat rows and second dim looks like bands => (rows, bands, cols)
            if cube.shape[0] == lat.shape[0] and cube.shape[2] == lat.shape[1]:
                return np.transpose(cube, (1, 0, 2))  # -> (bands, rows, cols)
            # if already (bands, rows, cols)
            if cube.shape[1] == lat.shape[0] and cube.shape[2] == lat.shape[1]:
                return cube
            # fallback: try detect bands dimension as the one not matching lat rows/cols
            raise ValueError(f"Cube shape {cube.shape} doesn't match lat grid {lat.shape}")

        VNIR_brc = to_brc(VNIRcube)
        SWIR_brc = to_brc(SWIRcube)

        dataReshaped = np.concatenate([VNIR_brc, SWIR_brc], axis=0).astype(np.float32)
        bands, rows, cols = dataReshaped.shape
        print("Stacked shape (bands, rows, cols):", dataReshaped.shape)

        # --- crude affine geotransform from lat/lon (OK for quick-look; not perfect for L1) ---
        xmin, xmax = float(lon.min()), float(lon.max())
        ymin, ymax = float(lat.min()), float(lat.max())
        xres = (xmax - xmin) / cols
        yres = (ymax - ymin) / rows
        geotransform = (xmin, xres, 0, ymax, 0, -yres)

        # output
        out_name = filename[:-4] + "_stack.tif"  # safer than [:-3]
        driver = gdal.GetDriverByName("GTiff")
        output_raster = driver.Create(out_name, cols, rows, bands, gdal.GDT_Float32)

        # write bands (GDAL bands are 1-based)
        for b in range(bands):
            outband = output_raster.GetRasterBand(b + 1)
            outband.WriteArray(dataReshaped[b, :, :])

        # tag CRS as WGS84 (since geotransform is in degrees)
        output_raster.SetGeoTransform(geotransform)
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)
        output_raster.SetProjection(srs.ExportToWkt())

        output_raster.FlushCache()
        output_raster = None

        print("Wrote:", out_name)
        print("Conversion complete.")

In [12]:
###Put one or many files in a folder. Modify the folder path and run this part of the code:

In [5]:
#enter folder path with he5 PRISMA files in it
folderPath= r"C:\Users\smclaugh1\OneDrive - Freeport-McMoRan Inc\projects_2026\PRISMA\PRS_L2D_STD_20241213102717_20241213102721_0001"
listImages=[]
for file in os.listdir(folderPath):
      listImages.append(os.path.join(folderPath, file))
print ("he5 image list: ", listImages)

#apply function PRISMA2GeoTIFF
for filename in listImages:
    print("Processing image", filename)
    PRISMA2GeoTIFF(filename)
print ("All files processed.")

he5 image list:  ['C:\\Users\\smclaugh1\\OneDrive - Freeport-McMoRan Inc\\projects_2026\\PRISMA\\PRS_L2D_STD_20241213102717_20241213102721_0001\\PRS_L2D_STD_20241213102717_20241213102721_0001.he5']
Processing image C:\Users\smclaugh1\OneDrive - Freeport-McMoRan Inc\projects_2026\PRISMA\PRS_L2D_STD_20241213102717_20241213102721_0001\PRS_L2D_STD_20241213102717_20241213102721_0001.he5
Using swath: HDFEOS/SWATHS/PRS_L2D_HCO
SWIR path: HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/SWIR_Cube shape: (1230, 57, 1313)
VNIR path: HDFEOS/SWATHS/PRS_L2D_HCO/Data Fields/VNIR_Cube shape: (1230, 22, 1313)
LAT/LON paths: HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Latitude HDFEOS/SWATHS/PRS_L2D_HCO/Geolocation Fields/Longitude shapes: (1230, 1313) (1230, 1313)
Stacked shape (bands, rows, cols): (79, 1230, 1313)


c:\Users\smclaugh1\AppData\Local\anaconda3\Lib\site-packages\osgeo\gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Wrote: C:\Users\smclaugh1\OneDrive - Freeport-McMoRan Inc\projects_2026\PRISMA\PRS_L2D_STD_20241213102717_20241213102721_0001\PRS_L2D_STD_20241213102717_20241213102721_0001_stack.tif
Conversion complete.
All files processed.
